# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 10 2023 8:55AM,259061,10,PRF-55163,Bio-PRF,Released
1,Mar 10 2023 8:55AM,259061,10,PRF-55167,Bio-PRF,Released
2,Mar 10 2023 8:55AM,259061,10,PRF-55170,Bio-PRF,Released
3,Mar 10 2023 8:55AM,259061,10,PRF-55177,Bio-PRF,Released
4,Mar 10 2023 8:55AM,259061,10,PRF-55183,Bio-PRF,Released
5,Mar 10 2023 8:55AM,259061,10,PRF-55187,Bio-PRF,Released
6,Mar 10 2023 8:30AM,259053,10,MTR-23-0294,"Senseonics, Incorporated",Released
7,Mar 10 2023 8:07AM,259059,22,643414,"NBTY Global, Inc.",Executing
8,Mar 10 2023 7:53AM,259058,19,ADV80013529,"AdvaGen Pharma, Ltd",Released
9,Mar 10 2023 7:53AM,259058,19,ADV80013530,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,259056,Released,1
16,259057,Released,34
17,259058,Released,7
18,259059,Executing,1
19,259061,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259056,NaN,NaN,1.0
259057,NaN,NaN,34.0
259058,NaN,NaN,7.0
259059,NaN,1.0,NaN
259061,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258997,25.0,17.0,1.0
259011,0.0,0.0,1.0
259013,0.0,0.0,1.0
259014,0.0,0.0,1.0
259018,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258997,25,17,1
259011,0,0,1
259013,0,0,1
259014,0,0,1
259018,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258997,25,17,1
1,259011,0,0,1
2,259013,0,0,1
3,259014,0,0,1
4,259018,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258997,25,17,1
1,259011,,,1
2,259013,,,1
3,259014,,,1
4,259018,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 10 2023 8:55AM,259061,10,Bio-PRF
6,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated"
7,Mar 10 2023 8:07AM,259059,22,"NBTY Global, Inc."
8,Mar 10 2023 7:53AM,259058,19,"AdvaGen Pharma, Ltd"
15,Mar 10 2023 7:41AM,259057,19,"AdvaGen Pharma, Ltd"
49,Mar 9 2023 4:11PM,259056,16,Sartorius Stedim Filters Inc.
50,Mar 9 2023 4:03PM,259054,19,"GCH Granules USA, Inc."
77,Mar 9 2023 3:33PM,259052,21,"NBTY Global, Inc."
78,Mar 9 2023 3:22PM,259046,16,Sartorius Stedim Filters Inc.
79,Mar 9 2023 2:50PM,259042,19,Eli Lilly and Company


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 10 2023 8:55AM,259061,10,Bio-PRF,,,6
1,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated",,,1
2,Mar 10 2023 8:07AM,259059,22,"NBTY Global, Inc.",,1,
3,Mar 10 2023 7:53AM,259058,19,"AdvaGen Pharma, Ltd",,,7
4,Mar 10 2023 7:41AM,259057,19,"AdvaGen Pharma, Ltd",,,34
5,Mar 9 2023 4:11PM,259056,16,Sartorius Stedim Filters Inc.,,,1
6,Mar 9 2023 4:03PM,259054,19,"GCH Granules USA, Inc.",,,27
7,Mar 9 2023 3:33PM,259052,21,"NBTY Global, Inc.",,,1
8,Mar 9 2023 3:22PM,259046,16,Sartorius Stedim Filters Inc.,,1,
9,Mar 9 2023 2:50PM,259042,19,Eli Lilly and Company,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 8:55AM,259061,10,Bio-PRF,6,,
1,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated",1,,
2,Mar 10 2023 8:07AM,259059,22,"NBTY Global, Inc.",,1,
3,Mar 10 2023 7:53AM,259058,19,"AdvaGen Pharma, Ltd",7,,
4,Mar 10 2023 7:41AM,259057,19,"AdvaGen Pharma, Ltd",34,,
5,Mar 9 2023 4:11PM,259056,16,Sartorius Stedim Filters Inc.,1,,
6,Mar 9 2023 4:03PM,259054,19,"GCH Granules USA, Inc.",27,,
7,Mar 9 2023 3:33PM,259052,21,"NBTY Global, Inc.",1,,
8,Mar 9 2023 3:22PM,259046,16,Sartorius Stedim Filters Inc.,,1,
9,Mar 9 2023 2:50PM,259042,19,Eli Lilly and Company,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 8:55AM,259061,10,Bio-PRF,6,,
1,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated",1,,
2,Mar 10 2023 8:07AM,259059,22,"NBTY Global, Inc.",,1,
3,Mar 10 2023 7:53AM,259058,19,"AdvaGen Pharma, Ltd",7,,
4,Mar 10 2023 7:41AM,259057,19,"AdvaGen Pharma, Ltd",34,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 8:55AM,259061,10,Bio-PRF,6.0,NaN,NaN
1,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Mar 10 2023 8:07AM,259059,22,"NBTY Global, Inc.",NaN,1.0,NaN
3,Mar 10 2023 7:53AM,259058,19,"AdvaGen Pharma, Ltd",7.0,NaN,NaN
4,Mar 10 2023 7:41AM,259057,19,"AdvaGen Pharma, Ltd",34.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 8:55AM,259061,10,Bio-PRF,6.0,0.0,0.0
1,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Mar 10 2023 8:07AM,259059,22,"NBTY Global, Inc.",0.0,1.0,0.0
3,Mar 10 2023 7:53AM,259058,19,"AdvaGen Pharma, Ltd",7.0,0.0,0.0
4,Mar 10 2023 7:41AM,259057,19,"AdvaGen Pharma, Ltd",34.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1036138,9.0,0.0,0.0
12,518037,1.0,1.0,0.0
15,258997,1.0,17.0,25.0
16,518102,1.0,1.0,0.0
19,1554257,70.0,1.0,0.0
21,518072,2.0,0.0,0.0
22,259059,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1036138,9.0,0.0,0.0
1,12,518037,1.0,1.0,0.0
2,15,258997,1.0,17.0,25.0
3,16,518102,1.0,1.0,0.0
4,19,1554257,70.0,1.0,0.0
5,21,518072,2.0,0.0,0.0
6,22,259059,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,9.0,0.0,0.0
1,12,1.0,1.0,0.0
2,15,1.0,17.0,25.0
3,16,1.0,1.0,0.0
4,19,70.0,1.0,0.0
5,21,2.0,0.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,9.0
1,12,Released,1.0
2,15,Released,1.0
3,16,Released,1.0
4,19,Released,70.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,0.0,0.0,25.0,0.0,0.0,0.0,0.0
Executing,0.0,1.0,17.0,1.0,1.0,0.0,1.0
Released,9.0,1.0,1.0,1.0,70.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,25.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,17.0,1.0,1.0,0.0,1.0
2,Released,9.0,1.0,1.0,1.0,70.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,25.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,17.0,1.0,1.0,0.0,1.0
2,Released,9.0,1.0,1.0,1.0,70.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()